In [ ]:
import requests # για να φέρνουμε τον html κώδικα των ιστοσελίδων
from bs4 import BeautifulSoup # για να βρίσκουμε μέσα στον html κώδικα, τα περιεχόμενα που μας ενδιαφέρουν
import pandas as pd # για να βάζουμε τα δεδομένα που συλλέγουμε σε dataframes

import time # για να κάνουμε παύσεις ανάμεσα στα αιτήματα που κάνουμε στον server του site και να μην τον ρίξουμε ή να μην μας αποκλείσει
import random # για να δημιουργούμε ποικιλία στον χρόνο των παύσεων ώστε τα αιτήματα στον server να μοιάζουν με ανθρώπινα και όχι μηχανικά

In [ ]:
def get_webpage_soup(url):
  # Ορισμός των headers για το requests
  headers = {
        "User-Agent": "Mozilla/5.0"
    }
  # Αίτημα στο server να φέρει τον html κώδικα από το url
  response = requests.get(url, headers=headers)

  # Έλεγχος της κατάστασης της σελίδας
  if response.status_code != 200:
    print(f"Error: {response.status_code}")
    return None
  else:
    # Μετατροπή του πηγαίου κώδικα της ιστοσελίδας σε αναζητήσιμο κείμενο (soup)
    print(f"{url} was scraped successfully")
    return BeautifulSoup(response.text, 'html.parser')

In [ ]:
def get_teaser_elements_from_articlesL(articlesL):
  articles_dataL = [] # κενή λίστα, στην οποία θα κάνουμε append() το dictionary με τα data κάθε άρθρου
  for article in articlesL:
    # title
    try:
      title = article.find("a", {"class": "py-4 mainlink"}).text.strip() # στην περίπτωση epikairothta
    except:
      try:
        title = article.find("p", class_=lambda c: c and "title is-5 medium_title" in c).text.strip() # στην περίπτωση κατηγοριών και αναζήτησης
      except:
        title=""
    # article url
    try:
      article_url = article.find("div", {"class": "content"}).a["href"] # στην περίπτωση κατηγοριών και αναζήτησης
    except:
      try:
        article_url = article.find("a", {"class": "py-4 mainlink"})["href"] # στην περίπτωση epikairothta
      except:
        article_url=""
    # date & time
    try:
      datetime_str = article.find("time")["datetime"] # στην περίπτωση epikairothta
      date_str = datetime_str.split("T")[0]
      time_str = datetime_str.split("T")[1].split("+")[0]
    except:
      try:
        datetime_str = article.find("div", {"class": "card-date"}).text.strip() # στην περίπτωση κατηγοριών και αναζήτησης
        dt_partsL = datetime_str.split(".")
        date_str = f"{dt_partsL[2]}-{dt_partsL[1]}-{dt_partsL[0]}"
        time_str = ""
      except:
        date_str = ""
        time_str = ""
    # section
    try:
      section = article.find("p", class_=lambda c: c and "subtitle" in c).text.strip()
    except:
      section = ""
    # excerpt
    try:
      excerpt = article.find("div", class_= lambda c: c and "lead" in c).text.strip()
    except:
      excerpt = ""
    # author
    try:
      author = article.find("span", {"class":"mb-2 author-link"}).text.strip()
    except:
      author = ""
    # Δημιουργία λεξικού με τα data του άρθρου
    articleD = {"title": title, "article_url": article_url, "date": date_str, "time": time_str, "section": section, "excerpt": excerpt, "author": author}

    # Προσθήκη των data του κάθε άρθρου στη λίστα των data των άρθρων
    articles_dataL.append(articleD)

  return articles_dataL

In [ ]:
url = "https://www.kathimerini.gr/search/%CF%84%CE%B5%CE%BC%CF%80%CE%B7/"
soup = get_webpage_soup(url)

https://www.kathimerini.gr/search/%CF%84%CE%B5%CE%BC%CF%80%CE%B7/ was scraped successfully


In [ ]:
def scrape_kathimerini_teaser_pages(scrape_url, start_page_nbr=1, end_page_nbr=1):
  # Κενή αρχική λίστα
  all_articles_dataL = []

  print(f"\rScraping started", end="")

  for page_nbr in range(start_page_nbr, end_page_nbr+1): # βάζω +1 για να σκραπάρω και τον αριθμό του τελευταίου page που έχω ορίσει
    # η δομή των teasers pages urls του kathmerini.gr
    if "search" in scrape_url:
      search_term = scrape_url.split("/")[4]
      if len(scrape_url.split("/")) > 7:
        search_params = scrape_url.split("/")[-1]
      else:
        search_params = ""
      url = f'https://www.kathimerini.gr/search/{search_term}/page/{page_nbr}/{search_params}'
    else:
      category = scrape_url.split("/")[3]
      url = f"https://www.kathimerini.gr/{category}/page/{page_nbr}/"

    print(f"\rScraping page {page_nbr}: {url}" + " "*2, end="")

    # φέρνω τη σούπα της σελίδας με το function που έφτιαξα παραπάνω στο βήμα 2
    soup = get_webpage_soup(url)

    # βάζω τον κώδικα με τον οποίο βρίσκω όλα τα blocks των άρθρων - το έφτιαξα στο βήμα 6
    articlesL = soup.find_all("div", {"class":"nx-article"})

    # εξορύσσω τα δεδομένα όλων των άρθρων της σελίδας με το function που έφτιαξα στο βήμα 8
    page_articles_dataL = get_teaser_elements_from_articlesL(articlesL)

    # προσθέτω τα δεδομένα των άρθρων της σελίδας στη λίστα με τα δεδομένα όλων των άρθρων - τη δημιούργησα στην αρχή αυτού του function
    all_articles_dataL.extend(page_articles_dataL)

    # Τυχαία καθυστέρηση μεταξύ 1 και 3 δευτερολέπτων ανάμεσα στα page scrapes
    delay = random.uniform(1, 3)
    time.sleep(delay)

  # Δημιουργία dataframe από τη λίστα με τα λεξικά των δεδομένων κάθε άρθρου
  df = pd.DataFrame(all_articles_dataL)
  print(f"\rScraping completed. Fetshed {len(df)} articles.")

  return df

In [ ]:
kathimerini_tempi_teasers_df = scrape_kathimerini_teaser_pages("https://www.kathimerini.gr/search/%CF%84%CE%B5%CE%BC%CF%80%CE%B7/", 9, 22)
kathimerini_tempi_teasers_df

Scraping page 9: https://www.kathimerini.gr/search/%CF%84%CE%B5%CE%BC%CF%80%CE%B7/page/9/  https://www.kathimerini.gr/search/%CF%84%CE%B5%CE%BC%CF%80%CE%B7/page/9/ was scraped successfully
Scraping page 10: https://www.kathimerini.gr/search/%CF%84%CE%B5%CE%BC%CF%80%CE%B7/page/10/  https://www.kathimerini.gr/search/%CF%84%CE%B5%CE%BC%CF%80%CE%B7/page/10/ was scraped successfully
Scraping page 11: https://www.kathimerini.gr/search/%CF%84%CE%B5%CE%BC%CF%80%CE%B7/page/11/  https://www.kathimerini.gr/search/%CF%84%CE%B5%CE%BC%CF%80%CE%B7/page/11/ was scraped successfully
Scraping page 12: https://www.kathimerini.gr/search/%CF%84%CE%B5%CE%BC%CF%80%CE%B7/page/12/  https://www.kathimerini.gr/search/%CF%84%CE%B5%CE%BC%CF%80%CE%B7/page/12/ was scraped successfully
Scraping page 13: https://www.kathimerini.gr/search/%CF%84%CE%B5%CE%BC%CF%80%CE%B7/page/13/  https://www.kathimerini.gr/search/%CF%84%CE%B5%CE%BC%CF%80%CE%B7/page/13/ was scraped successfully
Scraping page 14: https://www.kathimerini.g

title  \
0                 Τέμπη: Νέα αυτοψία στη θέση Κουλούρι   
1    Τα Τέμπη στη Βουλή: Κυβέρνηση υπό πίεση,  ασυν...   
2    Τέμπη: Αντιπαράθεση στη Βουλή για τα επεισόδια...   
3    Επεισόδια στη Θεσσαλονίκη: Μολότοφ, χημικά και...   
4    Τέμπη: Επίθεση με πέτρες στον σταθμό του ΟΣΕ σ...   
..                                                 ...   
205  Τέμπη – Θεσσαλονίκη: Επεισόδια μετά τη συγκέντ...   
206      Χιλιάδες διαδηλωτές στο Σύνταγμα για τα Τέμπη   
207  Σύνταγμα: Επεισόδια μετά τη συγκέντρωση για τα...   
208  Διαδηλώσεις για τα Τέμπη σε Ελλάδα και εξωτερι...   
209  Διαδηλώσεις για τα Τέμπη σε πολλές πόλεις της ...   

                                           article_url        date time  \
0    https://www.kathimerini.gr/society/563500327/t...  2025-03-06        
1    https://www.kathimerini.gr/politics/563500285/...  2025-03-06        
2    https://www.kathimerini.gr/politics/563500153/...  2025-03-05        
3    https://www.kathimerini.gr/society/563500141/e...  2025-03-05        
4    https://www.kathimerini.gr/society/563500075/e...  2025-03-05        
..                                                 ...         ...  ...   
205  https://www.kathimerini.gr/society/563437195/t...  2025-01-26        
206  https://www.kathimerini.gr/society/reportaz/56...  2025-01-26        
207  https://www.kathimerini.gr/society/563437147/s...  2025-01-26        
208  https://www.kathimerini.gr/society/563437075/d...  2025-01-26        
209  https://www.kathimerini.gr/society/563437057/d...  2025-01-26        

        section                                            excerpt author  
0      ΚΟΙΝΩΝΙΑ  Παραγγελία προς την Πυροσβεστική από τον ειδικ...         
1         ΒΟΥΛΗ  Σε συνθήκες πολιτικής θύελλας κινείται από χθε...         
2      ΠΟΛΙΤΙΚΗ  Βουλευτές του ΠΑΣΟΚ, του ΣΥΡΙΖΑ, του ΚΚΕ, της ...         
3      ΚΟΙΝΩΝΙΑ  Οι οδομαχίες σημειώνονται επί της οδού Εγνατία...         
4      ΚΟΙΝΩΝΙΑ  Περίπου 20 άτομα με καλυμμένα τα χαρακτηριστικ...         
..          ...                                                ...    ...  
205    ΚΟΙΝΩΝΙΑ  Επεισόδια ξέσπασαν στη Θεσσαλονίκη και πιο συγ...         
206  ΑΣΤΥΝΟΜΙΚΟ  Με κεντρικό σύνθημα «Δικαιοσύνη», η συγκέντρωσ...         
207    ΚΟΙΝΩΝΙΑ  Μικρή ομάδα ατόμων επιτέθηκε σε αστυνομικούς π...         
208    ΚΟΙΝΩΝΙΑ  Τεράστια κινητοποίηση για την απονομή δικαιοσύ...         
209    ΚΟΙΝΩΝΙΑ  Δεκάδες συγκεντρώσεις προγραμματίζονται για σή...         

[210 rows x 7 columns]

# ΜΗΝ ΞΕΧΑΣΕΤΕ ΝΑ ΣΩΣΕΤΕ ΤΟ df ΣΕ csv

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
kathimerini_tempi_teasers_df.to_csv("/content/drive/MyDrive/Colab Notebooks/tempi-data-analysis.csv", index=False)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
kathimerini_tempi_teasers_df.to_csv('tempi_naftemporiki_teasers.csv', index=False)

In [ ]:
from google.colab import files
files.download('tempi_naftemporiki_teasers.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

articles scraper

In [ ]:
import requests # για να φέρνουμε τον html κώδικα των ιστοσελίδων
from bs4 import BeautifulSoup # για να βρίσκουμε μέσα στον html κώδικα, τα περιεχόμενα που μας ενδιαφέρουν
import pandas as pd # για να βάζουμε τα δεδομένα που συλλέγουμε σε dataframes

import time # για να κάνουμε παύσεις ανάμεσα στα αιτήματα που κάνουμε στον server του site ώστε να μην τον ρίξουμε ή να μην μας αποκλείσει
import random # για να δημιουργούμε ποικιλία στον χρόνο των παύσεων ώστε τα αιτήματα στον server να μοιάζουν με ανθρώπινα και όχι μηχανικά

from tqdm import tqdm # για να βάλω progress bar με το οποίο θα παρακολουθώ την πορεία του scraping

In [ ]:
def get_webpage_soup(url):
  # Ορισμός των headers για το requests
  headers = {
        "User-Agent": "Mozilla/5.0"
    }
  # Αίτημα στο server να φέρει τον html κώδικα από το url
  response = requests.get(url, headers=headers)

  # Έλεγχος της κατάστασης της σελίδας
  if response.status_code != 200:
    #print(f"Error: {response.status_code}")
    return None
  else:
    # Μετατροπή του πηγαίου κώδικα της ιστοσελίδας σε αναζητήσιμο κείμενο (soup)
    # print(f"{url} was scraped successfully")
    return BeautifulSoup(response.text, 'html.parser')

In [ ]:
def get_article_elements_from_soup(soup, article_url):
  # απομόνωση του block με το περιεχόμενο του άρθρου από όλη τη σούπα της σελίδας
  article_block = soup.find("article")
  # article url
  article_url = article_url # το παίρνω από το arguement του function
  # title
  try:
    title = article_block.find("h1").text.strip()
  except:
    try:
      title = soup.find("div", {"class": "container is-fullhd p-3"}).h1.text.strip() # προσθήκη για να πιάνει και τον τίτλο όταν βρίσκεται σε αυτό το div
    except:
      title=""
  # datetime
  try:
    datetime_str = article_block.find("time")["datetime"]
  except:
    try:
      datetime_str = soup.find("div", {"class": "container is-fullhd p-3"}).find("time")["datetime"] # προσθήκη για να πιάνει και το datetime όταν βρίσκεται σε αυτό το div
    except:
      datetime_str = ""
      date_str = ""
      time_str = ""
  if datetime_str != "":
    date_str = datetime_str.split("T")[0]
    time_str = datetime_str.split("T")[1].split("+")[0]
  # section
  try:
    section = article_block.find("span", class_=lambda c: c and "nx-single-category-title" in c).text.strip()
  except:
    section = ""
  # author
  try:
    author = article_block.find("span", class_=lambda c: c and "author" in c).text.strip()
  except:
    try:
      author = soup.find("div", {"class": "container is-fullhd p-3"}).find("a", {"rel": "author"}).text.strip() # προσθήκη
    except:
      author = ""
  # excerpt
  try:
    excerpt = article_block.find("div", {"class": "nx-excerpt pb-5"}).text.strip()
  except:
    try:
      excerpt = soup.find("div", {"class": "container is-fullhd p-3"}).find("div", {"class": "newsletter-description nx-excerpt my-5"}).text.strip() # προσθήκη
    except:
      excerpt=""
  # text body
  try:
    text_body_p_tagsL = article_block.find("div", {"id": "main-content"}).find_all("p")
    text_body_textsL = []
    for p_tag in text_body_p_tagsL:
      p_tag_text = p_tag.text.strip()
      text_body_textsL.append(p_tag_text)
    text_body = " ".join(text_body_textsL)
  except:
    text_body=""
  # feature image
  try:
    feat_img_link = article_block.find("div", {"class": "post-thumbnail"}).img["src"]
  except:
    feat_img_link=""
  # feature image caption
  try:
    image_caption = article_block.find("div", {"class": "img-caption mb-5"}).text.strip()
  except:
    image_caption=""

  # Δημιουργία λεξικού με τα data του άρθρου
  articleD = {"article_url": article_url, "title": title, "date": date_str, "time": time_str, "section": section, "author": author,
              "excerpt": excerpt, "text_body": text_body, "feat_img_link": feat_img_link, "feat_img_cation": image_caption}

  return articleD

In [ ]:
def scrape_kathimerini_articles_pages(kathimerini_tempi_teasers_df):
  # δημιουργία κενής λίστας στην οποία θα προσθέτουμε ένα-ένα τα λεξικά με τα δεδομένα των άρθρων που θα σκραπάρουμε
  all_articles_dataL = []
  # λούπα σε κάθε row του teasers_df για να παίρνουμε ένα-ένα τα urls των άρθρων και να το σκραπάρουμε
  # χρησιμοποιούμε το tqdm για να βλέπουμε σε ένα progress bar πού βρισκόμαστε και πόσος χρόνος απομένει ακόμα
  for idx, row in tqdm(kathimerini_tempi_teasers_df.iterrows(), total=kathimerini_tempi_teasers_df.shape[0], desc="Processing rows"):
    # παίρνουμε το url του άρθρου που θα σκραπάρουμε από τη στήλη "article_url" του row του teasers_df στο οποίο βρίσκεται η for loop
    article_url = row['article_url']
    # scrape της σελίδας του url του κάθε άρθρου με το function που φτιάξαμε στο βήμα 2
    soup = get_webpage_soup(article_url)
    # δημιουργία του λεξικού με τα δεδομένα του κάθε άρθρου με το function που φτιάξαμε στο βήμα 9
    articleD = get_article_elements_from_soup(soup, article_url)
    # προσθήκη του λεξικού με τα δεδομένα του κάθε άρθρου στη λίστα που δημιουργήσαμε στη αρχή
    all_articles_dataL.append(articleD)
    # καθυστέρηση ανάμεσα στα requests στο server
    delay = random.uniform(1, 3)
    time.sleep(delay)
  # δημιουργία dataframe από τη λίστα των λεξικών με τα δεδομένα κάθε άρθρου
  df = pd.DataFrame(all_articles_dataL)
  print(f"\nScraping completed. Fetshed {len(df)} articles.")
  return df

In [ ]:
kathimerini_tempi_teasers_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/tempi-data-analysis.csv")
kathimerini_tempi_teasers_df.head()

title  \
0               Τέμπη: Νέα αυτοψία στη θέση Κουλούρι   
1  Τα Τέμπη στη Βουλή: Κυβέρνηση υπό πίεση,  ασυν...   
2  Τέμπη: Αντιπαράθεση στη Βουλή για τα επεισόδια...   
3  Επεισόδια στη Θεσσαλονίκη: Μολότοφ, χημικά και...   
4  Τέμπη: Επίθεση με πέτρες στον σταθμό του ΟΣΕ σ...   

                                         article_url        date  time  \
0  https://www.kathimerini.gr/society/563500327/t...  2025-03-06   NaN   
1  https://www.kathimerini.gr/politics/563500285/...  2025-03-06   NaN   
2  https://www.kathimerini.gr/politics/563500153/...  2025-03-05   NaN   
3  https://www.kathimerini.gr/society/563500141/e...  2025-03-05   NaN   
4  https://www.kathimerini.gr/society/563500075/e...  2025-03-05   NaN   

    section                                            excerpt  author  
0  ΚΟΙΝΩΝΙΑ  Παραγγελία προς την Πυροσβεστική από τον ειδικ...     NaN  
1     ΒΟΥΛΗ  Σε συνθήκες πολιτικής θύελλας κινείται από χθε...     NaN  
2  ΠΟΛΙΤΙΚΗ  Βουλευτές του ΠΑΣΟΚ, του ΣΥΡΙΖΑ, του ΚΚΕ, της ...     NaN  
3  ΚΟΙΝΩΝΙΑ  Οι οδομαχίες σημειώνονται επί της οδού Εγνατία...     NaN  
4  ΚΟΙΝΩΝΙΑ  Περίπου 20 άτομα με καλυμμένα τα χαρακτηριστικ...     NaN

In [ ]:
kathimerini_tempi_articles_df = scrape_kathimerini_articles_pages(kathimerini_tempi_teasers_df)
kathimerini_tempi_articles_df

Processing rows: 100%|██████████| 210/210 [09:10<00:00,  2.62s/it]


Scraping completed. Fetshed 210 articles.


article_url  \
0    https://www.kathimerini.gr/society/563500327/t...   
1    https://www.kathimerini.gr/politics/563500285/...   
2    https://www.kathimerini.gr/politics/563500153/...   
3    https://www.kathimerini.gr/society/563500141/e...   
4    https://www.kathimerini.gr/society/563500075/e...   
..                                                 ...   
205  https://www.kathimerini.gr/society/563437195/t...   
206  https://www.kathimerini.gr/society/reportaz/56...   
207  https://www.kathimerini.gr/society/563437147/s...   
208  https://www.kathimerini.gr/society/563437075/d...   
209  https://www.kathimerini.gr/society/563437057/d...   

                                                 title        date      time  \
0                 Τέμπη: Νέα αυτοψία στη θέση Κουλούρι  2025-03-06  07:10:13   
1    Τα Τέμπη στη Βουλή: Κυβέρνηση υπό πίεση,  ασυν...  2025-03-06  06:56:52   
2    Τέμπη: Αντιπαράθεση στη Βουλή για τα επεισόδια...  2025-03-05  22:57:23   
3    Επεισόδια στη Θεσσαλονίκη: Μολότοφ, χημικά και...  2025-03-05  22:21:36   
4    Τέμπη: Επίθεση με πέτρες στον σταθμό του ΟΣΕ σ...  2025-03-05  21:15:18   
..                                                 ...         ...       ...   
205  Τέμπη – Θεσσαλονίκη: Επεισόδια μετά τη συγκέντ...  2025-01-26  15:16:00   
206      Χιλιάδες διαδηλωτές στο Σύνταγμα για τα Τέμπη  2025-01-26  14:42:46   
207  Σύνταγμα: Επεισόδια μετά τη συγκέντρωση για τα...  2025-01-26  14:22:15   
208  Διαδηλώσεις για τα Τέμπη σε Ελλάδα και εξωτερι...  2025-01-26  13:03:05   
209  Διαδηλώσεις για τα Τέμπη σε πολλές πόλεις της ...  2025-01-26  11:22:12   

      section                                             author  \
0    Κοινωνία                                  Γιάννης Σουλιώτης   
1    Πολιτική  Σταύρος Παπαντωνίου\n\nΑντώνης Αντζολέτος\nΔώρ...   
2    Πολιτική                                           Newsroom   
3    Κοινωνία                                           Newsroom   
4    Κοινωνία                                           Newsroom   
..        ...                                                ...   
205  Κοινωνία                                           Newsroom   
206  Ρεπορτάζ                                   Λουκάς Βελιδάκης   
207  Κοινωνία                                   Λουκάς Βελιδάκης   
208  Κοινωνία                 Λουκάς Βελιδάκης\n\nΣταύρος Τζίμας   
209  Κοινωνία                                           Newsroom   

                                               excerpt  \
0    Παραγγελία προς την Πυροσβεστική από τον ειδικ...   
1    Μητσοτάκης: Δεν το βάζω κάτω – Συμφωνία στο πα...   
2    Βουλευτές του ΠΑΣΟΚ, του ΣΥΡΙΖΑ, του ΚΚΕ, της ...   
3    Οι οδομαχίες σημειώνονται επί της οδού Εγνατία...   
4    Περίπου 20 άτομα με καλυμμένα τα χαρακτηριστικ...   
..                                                 ...   
205                                                      
206  Με κεντρικό σύνθημα «Δικαιοσύνη», η συγκέντρωσ...   
207  Μικρή ομάδα ατόμων επιτέθηκε σε αστυνομικούς π...   
208  Τεράστια κινητοποίηση για την απονομή δικαιοσύ...   
209                                                      

                                             text_body  \
0    Ο ειδικός εφέτης ανακριτής Λάρισας Σωτήρης Μπα...   
1    Σε συνθήκες πολιτικής θύελλας κινείται από χθε...   
2    Αντιπαράθεση, με αφορμή τα επεισόδια στο κέντρ...   
3    Σοβαρά επεισόδια σημειώνονται αυτή την ώρα στη...   
4    Αγνωστοι επιτέθηκαν με πέτρες στον σιδηροδρομι...   
..                                                 ...   
205  Επεισόδια ξέσπασαν στη Θεσσαλονίκη και πιο συγ...   
206  Χιλιάδες πολίτες κατέκλυσαν την πλατεία Συντάγ...   
207  Eπεισόδια σημειώθηκαν μετά τη συγκέντρωση διαμ...   
208  Βουβή διαμαρτυρία πολλών χιλιάδων ανθρώπων εξε...   
209  Δεκάδες συγκεντρώσεις προγραμματίζονται για σή...   

                                         feat_img_link  \
0    https://www.kathimerini.gr/wp-content/uploads/...   
1    https://www.kathimerini.gr/wp-content/uploads/...   


In [ ]:
kathimerini_tempi_articles_df.to_csv("/content/drive/MyDrive/Colab Notebooks/kathimerini_tempi_articles.csv", index=False)

In [ ]:
kathimerini_tempi_articles_df.to_csv('kathimerini_tempi_articles.csv', index=False)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#from google.colab import files
#files.download('kathimerini_tempi_articles.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>